# Phys 260: Python assignment header

### (1) Fill out the cell below.  
The cell below is a **code cell**.  Fill out your University of Michigan uniqname, then your name, and collaborators in the cell below **inside the quotes**.  

**Do not delete the quotes.**  We will use this information to organize your assignments.  To edit and execute cells, double click inside the cell, type, and press \<shift\>+\<enter\> to execute.

In [ ]:
UNIQNAME = ""
NAME = ""
COLLABORATORS = ""

### (2) Check your python version.  
**Execute the cell below** (double click in the cell and press \<shift\>+\<enter\>, or click in the cell and press the Run button) to check that you are using a version of python that is compatible with the tool we are using to grade your assignments.  If your ```IPython``` version is too old, we will *not* be able to grade your assignments.

In [ ]:
import IPython
assert IPython.version_info[0] >= 3, "Your version of IPython is too old, please update it."

### (3) Do your best to answer all questions in the assignment.  
To answer questions, **replace** anything that says either
- "YOUR ANSWER HERE" 
- 
```
YOUR CODE HERE
raise NotImplementedError
``` 

with your answer/code.  Cells with either of the two bullet points above are cells of the notebook that will be graded.

**To edit markdown** cells (e.g. this one),  *double click in the cell to type*.  Press \<shift\>+\<enter\> to execute the cell.   

[first name] [last name], uniqname


### (4) Make sure your notebook runs sequentially.
After you complete this assignment, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).


# Physics 260 Python Tutorial/HW 6: Simulating charged particles in fields (30 points total)

## Tutorial Summary
- Replicate what you did in the preflight with less code in `__main__`
- The Runge-Kutta Method and effect on kinetic energy

## HW Summary
- How changes in the field affect the particle trajectory
- Simulating a velocity filter
- Comparisons of the energy budget using the Runge-Kutta Method

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import sys

#Here we are importing our own custom modules from files in our path. Especially for more complex projects,
#it is a good idea to put custom functions and classes you use into modules (.py files that are in your path)

import charge_particle_in_fields as cp
import electric_and_magnetic_fields as eb_methods

### Examine module contents (2 points -- 5 minutes in groups)

Below, we examine the module contents using the built-in `help` function.  List what classes and/or functions, etc. you find in the module `cp`, and what you find in the module `eb_methods`.

YOUR ANSWER HERE

In [ ]:
# Use this cell for executing help on the newly imported modules.
help(cp.ChargedParticle)

### Use the objects from the modules to do what you did in the preflight (5 points -- 5 minutes in groups)

Recall, in the preflight, you had to create an instance of `ChargedParticle`, time evolve the object with the method `evolve_particle`, then plot.  In the cell below, you will do the same but use the objects already defined in the modules you have imported.  

- First, define an instance of the `ChargedParticle` class with name `charged_particle`.  Remember that when you use `numpy` tools, you needed the alias `np` as a prefix for all tools.  Similarly, you will need either `cp` or `eb_methods` as a prefix.  Also, you will need to specify all parameters to the `ChargedParticle` class (see documentation for the class using help), including `x0`, `v0`, `efield_method`, and `bfield_method`. Take the initial position and velocity to be the origin and (0,0,0).  Use the `constant_electric_field` and `constant_magnetic_field` as your input methods.
- Next, evolve the object for 1000 timesteps with `dt` of 0.01.  Note, you'll also need to specify the `rate_of_change` method.  In the preflight, we used `get_rates_of_change_euler`.

In [ ]:
# Define the instance charged_particle here, with the same inputs as you used for the preflight and time evolve.
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
"""Execute to check you're on the right track"""
assert(np.abs(np.linalg.norm(charged_particle.x_vals[-1]) - 10.7202) < 0.1)
assert(charged_particle.efield_method == eb_methods.constant_electric_field)

Now, we make the same 3-d plot where we see the cycloidal motion of a charged particle in crossed electric and magnetic fields.

In [ ]:
# Now we make the same 3-d plot

#  Make your 3-d trajectory plot here
fig = plt.figure(figsize=(10,10)) 
ax = plt.gca(projection='3d')

# Access the particle positions here
particle_positions = np.array(charged_particle.x_vals)
print(particle_positions.shape)                     
ax.plot(particle_positions[:,0], particle_positions[:,1], particle_positions[:,2],lw=2)

ax.set_xlabel('x',fontsize='xx-large')
ax.set_ylabel('y',fontsize='xx-large')
ax.set_zlabel('z',fontsize='xx-large')
ax.view_init(30,-75)

## Runge-Kutta method:  Discuss in class
Below is a function that does the next-best-thing to the Euler method.  This is the Runge-Kutta method, that essentially uses values in between sampled points to get a better updated value.  You may remember approximating integrals (area under a curve) with right and left sum, then using the mid-point, then finally using the trapezoid method.  That is essentially what we are doing here: https://www.khanacademy.org/math/ap-calculus-ab/ab-integration-new/ab-6-2/a/riemann-sums-review

The Runge-Kutta method does the following:

- Use the acceleration due to the fields at the current position to figure out a $\vec{v}_{i+1/2}=\vec{v}_{i}+\Delta\vec{v}_{i+1/2}$, or the change in velocity over half a time-step (e.g. between timestep $i$ and timestep $i+1$).

- Use the updated velocity (over the half a time-step) to find the new particle position in half a time-step, $\vec{x}_{i+1/2}= \vec{x}_i+\Delta\vec{x}_{i+1/2}$.  

- Use the electric and magnetic fields at $\vec{x}_{i+1/2}$ for a particle with velocity $\vec{v}_{i+1/2}$ to calculate the half time-step acceleration, $\vec{a}_{i+1/2}$ that will be used to actually update the velocity for $\vec{v}_{i+1}$.  We also use the half time-step velocity $\vec{v}_{i+1/2}$ to update the position for $\vec{x}_{i+1}$.

In [ ]:
def get_rates_of_change_runge_kutta(particle_position, particle_velocity, electric_field_function,
                                        magnetic_field_function, q, m, dt) :
    ''' This uses the Runge-Kutta method to calculate the updated values.  
    
    This method relies on mid-points. Note, this method uses the same arguments and has the same output types as 
    get_rates_of_change_euler.
    
    '''

    current_velocity, current_acceleration = cp.get_rates_of_change_euler(particle_position, particle_velocity, electric_field_function,
                                        magnetic_field_function, q, m, dt)
    
    mid_particle_velocity = current_velocity + current_acceleration * dt/2
    mid_particle_position = particle_position + mid_particle_velocity * dt/2
    
    # Calculate the acceleration due to the electric and magnetic fields at the mid_particle_position
    runge_kutta_velocity, runge_kutta_acceleration = cp.get_rates_of_change_euler(mid_particle_position, mid_particle_velocity,
                                                                        electric_field_function,
                                                                        magnetic_field_function, q, m, dt)
    return runge_kutta_velocity, runge_kutta_acceleration

Below we look at the call signature of `get_rates_of_change_euler`.  Take a look at how this compares with `get_rates_of_change_runge_kutta`, defined above.  Quick note, the function above makes use of `get_rates_of_change_euler`, defined in the `cp` module.

In [ ]:
help(cp.get_rates_of_change_euler)

### Use the Runge-Kutta function (3 points)

Create another instance of a charged particle with name `charged_particle_rk` whose initial state is teh same as `charge_particle`.  Now, evolve `charged_particle_rk` for the same number of timesteps and `dt`, but use `get_rates_of_change_runge_kutta` as the `rate_of_change_method`.

In [ ]:
# Define the instance charged_particle here, with the same inputs as you used for the preflight and time evolve.
# YOUR CODE HERE
raise NotImplementedError()
print(len(charged_particle_rk.v_vals))

In [ ]:
"""Execute to check you're on the right track"""
assert(np.abs(np.linalg.norm(charged_particle_rk.x_vals[-1]) - 10.686) < 0.01)

### Make the 3-d trajectory plot again

Plot the 3-d trajectory for `charged_particle_rk`.  You should only need to change one line from the previous plot, and you will notice that there is no perceptible difference in the visual trajectories.

In [ ]:
# Now we make the same 3-d plot

#  Make your 3-d trajectory plot here
fig = plt.figure(figsize=(10,10)) 
ax = plt.gca(projection='3d')

# Access the particle positions here
particle_positions = np.array(charged_particle_rk.x_vals)
print(particle_positions.shape)                     
ax.plot(particle_positions[:,0], particle_positions[:,1], particle_positions[:,2],lw=2)

ax.set_xlabel('x',fontsize='xx-large')
ax.set_ylabel('y',fontsize='xx-large')
ax.set_zlabel('z',fontsize='xx-large')
ax.view_init(30,-75)

### Calculate the kinetic energy (3 points)

Calculate the kinetic energy of `charged_particle` and `charged_particle_rk`, `ke_charged_particle_rk` and `ke_charged_particle` respectively.  Hint: You may find it useful to have the particle velocities as an array (lists can be "turned into" arrays) and the `np.linalg.norm` function may be useful to you.

In [ ]:
# Define ke_charged_particle and ke_charged_particle_rk
ke_charged_particle =0.5 * np.linalg.norm(np.array(charged_particle.v_vals),axis=1)**2 #1/2 m v^2
ke_charged_particle_rk =0.5 * np.linalg.norm(np.array(charged_particle_rk.v_vals),axis=1)**2

In [ ]:
"""Execute to check you're on the right track"""
assert((ke_charged_particle[0] == 0))
assert((ke_charged_particle_rk[0] == 0))
assert(ke_charged_particle.size == len(charged_particle.v_vals))
assert(ke_charged_particle.size == charged_particle.timesteps.size)

### Plot the kinetic energy over time (3 points)

Plot on three axes: 
- The kinetic energy of `charged_particle`, whose trajectory calculation used the Euler method, as a function of time, 
- The kinetic energy of `charged_particle_rk`, whose trajectory calculation used the Runge-Kutta method, as a function of time, 
- The ratio between the second to the first. 

You'll notice a dip in the last one - this is because values are close to zero, and one is much closer to zero than the other at the same timestep.  I suggest trying to set the y axis of the third axis to logscale and to try plotting every 5 points.  If you have an array, `test_array = np.arange(10)`, then `test_array[::5]` gives you `np.array([0,5])`.  Array slicing has the form of `start:stop:step`.  Note, `test_array[1::5]` gives you `np.array([1,6])`.  


Full credit for labeled and legible axes.

In [ ]:
# Example showing the step of array slicing
test_array = np.arange(10)
print(test_array[::5])
print(test_array[1::5])

In [ ]:
# Make your three axis plot here
# YOUR CODE HERE
raise NotImplementedError()

### Talk through the figure (2 points)

Talk through the figure in the space below as though you were writing a caption.  You'll notice that even outside of the spike, the ratio of the kinetic energies is not quite 1:1.  Incorporate what you know about the different methods to calculate the positions and velocities at the next timestep (Euler vs. Runge-Kutta). (Hint: One is more accurate than the other.)

YOUR ANSWER HERE

Also, this might be a good time to qualitatively walk through the particle's behavior (cycloid). There is no magnetic force at $t=0$ since the charge starts at rest.  The electric force accelerates the charge along the x-axis, leading to a non-zero velocity in the next timestep. The particle now has a velocity in the x-direction, perpendicular to the y-direction of the magnetic field, leading to a magnetic force that first accelerates the particle in the z-direction, then continues to rotate the particle in the xz plane about a center on the z axis.  

The magnetic force though draws the charged particle away from x-axis along a curved path. The force due to the magnetic field is opposed by the force from the electric field in the x-direction. The velocity of charged particle ultimately reduces to zero at z-axis. This cycle forms a cycloid motion, similar to a point on a bicycle wheel that is rolling without slipping.

In [ ]:
# Example plot
plt.plot(np.array(charged_particle.x_vals)[:,0], np.array(charged_particle.x_vals)[:,2])
plt.xlabel('x position',fontsize='xx-large')
plt.ylabel('z position',fontsize='xx-large')
plt.title('Cycloid motion',fontsize='xx-large')

##  Homework:  Particle trajectory in different fields (12 points total)

- A brief introduction to lambda functions
- Particle in a parallel E and B field (4 points total)
- The velocity filter (2 points)
- Motion in non-uniform magnetic field (2 points)
- Energy comparison (2 points)

 ### Using lambda functions (a short cut... sort of)

For new E and B field configurations, we need to have the appropriate functions to provide as parameters to the `ChargedParticle` class (i.e. the kwargs `efield_method` and `bfield_method`.  We actually have two options here.  We can either keep adding new functions in the `electric_and_magnetic_fields` module, or we can shorthand with a [lambda function](https://realpython.com/python-lambda/). See the link for more details and/or take a look at the following examples. 

An example of this would be replacing `eb_methods.constant_electric_field` with `lambda x : np.array([1,0,0])`.  You can swapping the two in the tutorial above.  Note - the tests will fail, but you can still make the figure.  Switch back to `eb_methods.constant_electric_field` after you try it out.

Another example of this from the preflight would be to replace the original function definition of `linearly_increasing_bfield` with 
```
linearly_increasing_bfield = lambda x : np.array([0,0,0.1*x[2]])
```

In the latter case, we have given a name to the lambda function.  In the former, the lambda function has the name `efield_method` when used in the class.  Below is an example use of the latter.

In [ ]:
#  Example use of a lambda function
linearly_increasing_bfield = lambda x : np.array([0,0,0.1*x[2]])

linearly_increasing_bfield(np.array([5,6,7]))

## Parallel E and B fields (2 points)

You already have a constant electric field function pointing in the x direction.  You will now need a constant magnetic field that is `np.array([1,0,0])`.

Next, create an instance of the `ChargedParticle` class with name `cp_in_parallel_fields` with the appropriate parameter inputs.  Everything should be the same as `charged_particle` with the exception of the magnetic field method (use a lambda function) and an initial velocity to be perpendicular to both fields, i.e. `v0=np.array([0,1,0])`.  Time evolve as before with 10000 timesteps, a `dt` of 0.01, and use Euler's method.

In [ ]:
# Define cp_in_parallel_fields here
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
"""Execute to check you're on the right track"""
assert(np.abs(np.linalg.norm(cp_in_parallel_fields.v_vals[-1]) - 100) < 0.0036)

In [ ]:
# Now we make the same 3-d plot
#  Make your 3-d trajectory plot here
fig = plt.figure(figsize = (10,10)) 
ax = plt.gca(projection='3d')

# Access the particle positions here
particle_positions = np.array(cp_in_parallel_fields.x_vals)
print(particle_positions.shape)                     
ax.plot(particle_positions[:,0], particle_positions[:,1], particle_positions[:,2])

ax.set_xlabel('x',fontsize='xx-large')
ax.set_ylabel('y',fontsize='xx-large')
ax.set_zlabel('z',fontsize='xx-large')
ax.view_init(30,-50)

### Double the fields (2 points)

Use the appropriate lambda functions to create instances of `ChargedParticle`: 
- `cp_parallel_2e_b`, where the electric field is `np.array([2,0,0])` (twice the magnitude of the magentic field
- `cp_parallel_e_2b`, where the magnetic field is `np.array([2,0,0])` (twice the magnitude of the electric field

Evolve each with the same arguments as the original case.

In [ ]:
# Define the instances here
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
"""Execute to check you're on the right track"""
assert(np.abs(np.linalg.norm(cp_parallel_2e_b.x_vals[-1])-9997) < 0.001 )
assert(np.abs(np.linalg.norm(cp_parallel_e_2b.x_vals[-1])-4998.501) < 0.001 )

In [ ]:
#  Make your 3-d trajectory plot here
fig = plt.figure(figsize=(12,12)) 
ax = plt.gca(projection='3d')

cparts = [cp_in_parallel_fields, cp_parallel_2e_b, cp_parallel_e_2b]
labels = ['E=B', '2E', '2B']
# Access the particle positions here
for cpart, label in zip(cparts, labels) : 
    particle_positions = np.array(cpart.x_vals)
    print(particle_positions.shape)                     
    ax.plot(particle_positions[:,0], particle_positions[:,1], particle_positions[:,2], label=label,lw=1.5)

ax.legend(fontsize='large')
ax.set_xlabel('x',fontsize='xx-large')
ax.set_ylabel('y',fontsize='xx-large')
ax.set_zlabel('z',fontsize='xx-large')
ax.view_init(30,-35)

### Talk through the figure (1 point)

Explain what happens to the particle's path as you increase E and increase B.  You may want to play around with the viewing angle.

YOUR ANSWER HERE

### The velocity filter

Consider a particle with charge 1 C and mass 1 kg moving with velocity $v=1\hat{x}$ m/s. If this particle enters a region with a magnetic field $B = 1\hat{y}$T, it will experience a force. You can calculate this by hand. You could then create an electric field to perfectly oppose this magnetic force. If you set up this field configuration you would have a "velocity filter."

In the cell below, instantiate charged particle objects (2 point)
- Create an instance of `ChargedParticle` with name `cp_vfilter_pass` that is instantiated with the appropriate conditions to pass through.
- Create an instance with name `cp_vfilter_fast` whose velocity is twice that of the first.
- Create an instance with name `cp_vfilter_slow` whose velocity is half that of the first.
- Time evolve all with 10 steps, dt of 0.01.

In [ ]:
# Create instances of charged particles in a velocity selector
# YOUR CODE HERE
raise NotImplementedError()


In [ ]:
"""Execute to check you're on the right track"""
assert((cp_vfilter_pass.v_vals[-1] == np.array([1,0,0])).all())

### Fill out the plotting cell below (1 point)

Plot the passing particle trajectory, the fast particle, and the slow particle.  Label each "pass", "fast", and "slow". 

Note: for a more compact way of plotting multiple paths, look back at the previous plotting example which uses a for loop and the <a href="https://realpython.com/python-zip-function/">zip function</a> to iterate over all of the plots.

In [ ]:
#  Make your 3-d trajectory plot here
fig = plt.figure(figsize=(10,10)) 
ax = plt.gca(projection='3d')

# YOUR CODE HERE
raise NotImplementedError()
    
ax.legend(fontsize='large')
ax.set_xlabel('x',fontsize='xx-large')
ax.set_ylabel('y',fontsize='xx-large')
ax.set_zlabel('z',fontsize='xx-large')
ax.view_init(40,-65)

### Motion in non-uniform magnetic field

For this problem the magnetic field will not be uniform, instead it is going to be changing as a function of position.  The magnetic field in cylindrical coordinates is:
\begin{equation}
 B(r,\phi,z) = - \frac {B_0} {2} r \hat{r} + (B_0z+B_1)\hat{z}
\end{equation}

That is the the magnitude of the field is changing in $z$ and $r$, but is not changing in $\phi$.  Let $B_1=1$T and $B_0=.1$T/m.

Take the electric field to be zero.

As opposed to the contrived example in the preflight, this is in fact a field that obeys Maxwell's equations. Such a field could not really fill all space... it blows up as $r\rightarrow\infty$. However, this field could exist inside a long, cylindrical coil of wire (a solenoid) where the number of turns per meter is increasing linearly. Such fields are sometimes used at particle accelerators to guide beams of charged particles.

For the sake of coding this equation below, here is the field in cartesian coordinates:
$$
   \mathbf{B} = -\frac{B_0}{2}\left(x\hat{x} + y\hat{y} - 2z\hat{z}\right) + B_1\hat{z}
$$


## Model a charged particle in this field (2 point)
- Create a `ChargedParticle` instance with name `cp_in_b_solenoid` using most of the initial conditions as before, with `x0=np.array([1,0,0])`, `v0=np.array([0,0,1])`, zero electric field and the solenoid magnetic field described above.  Time evolve with Euler over 1000 timesteps, dt=0.01.
- Create an instance `cp_in_b_solenoid_rk` and time evolve with Runge-Kutta.  BUT, first copy the function `get_rates_of_change_runge_kutta` below `get_rates_of_change_euler`.  You will need to re-import (or reload) the modules every time you make a change to the `.py` file.  But, now you can use both functions from the imported module `cp`.

In [ ]:
# Define and evolve cp_in_b_solenoid and cp_in_b_solenoid_rk here
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
"""Execute to check you're on the right track"""
assert(np.abs(np.linalg.norm(cp_in_b_solenoid.v_vals[-1]) - 1) < 5e-4)
assert(np.abs(np.linalg.norm(cp_in_b_solenoid_rk.v_vals[-1]) - 1) < 1e-6)

### Plot solenoid paths (1 points)

Plot paths of the charged particles for both the Euler and Runge-Kutta methods

In [ ]:
#  Make your 3-d trajectory plot here
fig = plt.figure(figsize=(10,10)) 
ax = plt.gca(projection='3d')

# YOUR CODE HERE
raise NotImplementedError()
    
ax.legend(fontsize='large')
ax.set_xlabel('x',fontsize='xx-large')
ax.set_ylabel('y',fontsize='xx-large')
ax.set_zlabel('z',fontsize='xx-large')
ax.view_init(40,-65)

### Compare kinetic energies (1 point)

What is the ratio of final kinetic energies of the Euler vs. R-K calculation, `e2rk_ratio`?

In [ ]:
# Calculate e2rk_ratio here
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
"""Execute to check you're on the right track"""
assert(e2rk_ratio - 1 < .000968)